In [1]:
#imports
import pandas as pd
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical
from collections import Counter

/home/hitkul/anaconda3/envs/ps3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#loading data
reviwes = pd.read_csv('reviews.txt',header=None)
labels = pd.read_csv('labels.txt',header=None)

In [3]:
#counting word frequency to create vocabulary
counter_ = Counter()
for review in reviwes.values:
    for word in review[0].split(" "):
        counter_[word]+=1
# print(counter_)

In [4]:
print(len(counter_))

74074


In [5]:
#keeping only 10000 most commmon words, frequency of 10000th word can be used to see if we need ot increase or decrease this number
vocabulary = sorted(counter_,key=counter_.get,reverse = True)[:10000]

In [6]:
#word to index, will need this mapping to create word vector 
word_to_index = dict()
for index,word in enumerate(vocabulary):
    word_to_index[word] = index

In [7]:
def get_word_vector(data):
    vector = np.zeros(len(vocabulary), dtype=np.int_)
    for word in data.split(' '):
        index = word_to_index.get(word, None)
        if index != None:
            vector[index]+=1
    return vector

In [8]:
review_vectors = np.zeros((len(reviwes),len(vocabulary)),dtype=np.int_)
for index,(_,data) in enumerate(reviwes.iterrows()):
    review_vectors[index] = get_word_vector(data[0])

In [9]:
review_vectors[:5]

array([[ 18,   9,  27, ...,   0,   0,   0],
       [  5,   4,   8, ...,   0,   0,   0],
       [ 78,  24,  12, ...,   0,   0,   0],
       [167,  53,  23, ...,   0,   0,   0],
       [ 19,  10,  11, ...,   0,   0,   0]])

In [14]:
Y = (labels=='positive').astype(np.int_)
shuffle = np.arange(len(labels))
np.random.shuffle(shuffle)
test_train_split = 0.9
train_split, test_split = shuffle[:int(len(labels)*test_train_split)], shuffle[int(len(labels)*test_train_split):]
trainX, trainY = review_vectors[train_split,:], to_categorical(Y.values[train_split], 2)
testX, testY = review_vectors[test_split,:], to_categorical(Y.values[test_split], 2)
trainX = tflearn.data_utils.pad_sequences(trainX, maxlen=100, value=0.)
testX = tflearn.data_utils.pad_sequences(testX, maxlen=100, value=0.)

In [15]:
# Network building
tf.reset_default_graph()
net = tflearn.input_data([None, 100])
net = tflearn.embedding(net, input_dim=10000, output_dim=128)
net = tflearn.lstm(net, 128, dropout=0.8)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy')

In [16]:
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(trainX, trainY,n_epoch=1,validation_set=0.1, show_metric=True,batch_size=32)

Training Step: 632  | total loss: 1.38629 | time: 125.701s
| Adam | epoch: 001 | loss: 1.38629 - acc: 0.4980 -- iter: 20224/20250
Training Step: 633  | total loss: 1.38629 | time: 130.055s
| Adam | epoch: 001 | loss: 1.38629 - acc: 0.5263 | val_loss: 1.38629 - val_acc: 0.9396 -- iter: 20250/20250
--


In [21]:
(np.array(model.predict(testX))[:,0]>=0.5).astype(np.int_)

predictions = (np.array(model.predict(testX))[:,0] >= 0.5).astype(np.int_)
# test_accuracy = np.mean(predictions == testY[:,0], axis=0)
# print("Test accuracy: ", test_accuracy)

array([1, 1, 1, ..., 1, 1, 1])